In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import json
import os
import numpy as np
from utils import *
import math
import heapq
import random

In [31]:
fd_collection = getCollection("team_5_mwdb_phase_2", "fd_collection")
all_images = fd_collection.find()

In [32]:
selected_latent_space = valid_latent_spaces[
    str(input("Enter latent space - one of " + str(list(valid_latent_spaces.keys()))))
]

selected_feature_model = valid_feature_models[
    str(input("Enter feature model - one of " + str(list(valid_feature_models.keys()))))
]

k = int(input("Enter value of k: "))
if k < 1:
    raise ValueError("k should be a positive integer")

selected_dim_reduction_method = str(
    input(
        "Enter dimensionality reduction method - one of "
        + str(list(valid_dim_reduction_methods.keys()))
    )
)

image_id = int(input("Enter image ID: "))
if image_id < 0 and image_id > 8676 and image_id % 2 != 0:
    raise ValueError("image id should be even number between 0 and 8676")

img_label = all_images[int(image_id / 2)]["true_label"]

knum = int(input("Enter value of knum: "))
if knum < 1:
    raise ValueError("knum should be a positive integer")

match selected_latent_space:
    case "":
        if selected_dim_reduction_method == "lda":
            if os.path.exists(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-model.joblib") and os.path.exists(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json"):
                if os.path.exists(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-model.joblib"):
                    model = load(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-model.joblib")
                    data = json.load(open(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json"))
                    print(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-model.joblib and json loaded")
                else:
                    print(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-model.joblib does not exist")
            else:
                if os.path.exists(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json"):
                    data = json.load(open(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json"))
                    print(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json loaded")
                else:
                    print(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json does not exist")
    case "cp":
        if os.path.exists(f"{selected_feature_model}-cp-{k}-semantics.json"):
            data = json.load(open(f"{selected_feature_model}-cp-{k}-semantics.json"))
            print(f"{selected_feature_model}-cp-{k}-semantics.json loaded")
        else:
          print(f"{selected_feature_model}-cp-{k}-semantics.json does not exist")
    case _:
        if selected_dim_reduction_method == "lda":
            if os.path.exists(f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}-{k}-model.joblib") and os.path.exists(f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json"):
                model = load(f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}-{k}-model.joblib")
                data = json.load(open(f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json"))
                print(f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}-{k}-model.joblib loaded")
            else:
                print(f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}-{k}-model.joblib does not exist")
        else:
            if os.path.exists(f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}img-{k}-semantics.json"):
                data = json.load(open(f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json"))
                print(f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json loaded")
            else:
                print(f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json does not exist")

label_sim-cm_fd-lda-10-model.joblib loaded


In [33]:
def extract_similarities_ls1_ls4(latent_space, dim_reduction, selected_feature_model, data, image_id):

  image_fd = np.array(all_images[int(image_id / 2)][selected_feature_model]).flatten()

  match dim_reduction:

    case 'svd':
      U = np.array(data["image-semantic"])
      S = np.array(data["semantics-core"])
      if len(S.shape) == 1:
        S = np.diag(S)
      V = np.transpose(np.array(data["semantic-feature"]))
      
      comparison_feature_space = np.matmul(U, S)

      if latent_space == "image_sim":
        comparison_vector = comparison_feature_space[int(image_id / 2)]
      else:
        comparison_vector = np.matmul(np.matmul(image_fd, V), S)
    
    case "nmf":
      H = np.array(data['semantic-feature'])
      comparison_feature_space = np.array(data['image-semantic'])

      if latent_space == "image_sim":
        comparison_vector = comparison_feature_space[int(image_id / 2)]
      else:
        comparison_vector = np.matmul(image_fd, np.transpose(H))

    case "kmeans":
      comparison_vector = []
      comparison_feature_space = np.array(data["image-semantic"])
      S = np.array(data["semantic-feature"])

      for centroid in S:
        if latent_space == "image_sim":
          sim_matrix = np.array(data["sim-matrix"])
          comparison_vector.append(math.dist(sim_matrix[int(image_id / 2)], centroid))
        else:
          comparison_vector.append(math.dist(image_fd, centroid))
    
    case "lda":
      comparison_feature_space = np.array(data["image-semantic"])
      if latent_space == "image_sim":
        comparison_vector = comparison_feature_space[int(image_id / 2)]
      else:
        fd = np.array(all_images[int(image_id / 2)][selected_feature_model])
        min_value = np.min(fd)
        feature_vectors_shifted = fd - min_value
        comparison_vector = model.transform(feature_vectors_shifted.flatten().reshape(1, -1)).flatten()
        print(comparison_feature_space.shape)
        print(comparison_vector.shape)

  n = len(comparison_feature_space)

  distances = []
  for i in range(n):
    if (i * 2) != image_id:
      distances.append({"image_id": i, "label": all_images[i]["true_label"], "distance": math.dist(comparison_vector, comparison_feature_space[i])})

  distances = sorted(distances, key=lambda x: x["distance"], reverse=False)

  similar_labels = []
  unique_labels = set()

  for img in distances:
    if img['label'] not in unique_labels:
      similar_labels.append(img)
      unique_labels.add(img["label"])

      if len(similar_labels) == knum:
        break


  for x in similar_labels:
    print(x)

In [34]:
def extract_similarities_ls2(data, image_id):

  IS = np.array(data["image-semantic"])
  S = np.array(data["semantics-core"])

  if len(S.shape) == 1:
    S = np.diag(S)

  comparison_feature_space = np.matmul(IS, S)
  comparison_vector = comparison_feature_space[int(image_id / 2)]

  distances = []

  n = len(comparison_feature_space)
  for i in range(n):
    if i != (image_id / 2):
      distances.append({"image_id": i * 2, "label":  all_images[i]["true_label"], "distance": math.dist(comparison_vector, comparison_feature_space[i])})
  
  distances = sorted(distances, key=lambda x: x["distance"], reverse=False)

  similar_labels = []
  unique_labels = set()

  for img in distances:
    if img["label"] not in unique_labels and img["label"] != img_label:
      similar_labels.append(img)
      unique_labels.add(img["label"])

      if len(similar_labels) == knum:
        break


  for x in similar_labels:
    print(x)

In [41]:
def extract_similarities_ls3(dim_reduction, data, image_id):

  img_label = all_images[int(image_id / 2)]["true_label"]

  match dim_reduction:

    case 'svd':
      U = np.array(data["image-semantic"])
      S = np.array(data["semantics-core"])
      V = np.transpose(np.array(data["semantic-feature"]))

      comparison_feature_space = np.matmul(U, S)
      comparison_vector = comparison_feature_space[img_label]
    
    case "nmf":
      comparison_feature_space = np.array(data['image-semantic'])
      comparison_vector = comparison_feature_space[img_label]

    case "kmeans":
      comparison_feature_space = np.array(data["image-semantic"])
      comparison_vector = comparison_feature_space[img_label]

    case "lda":
      comparison_feature_space = np.array(data["image-semantic"])
      comparison_vector = comparison_feature_space[img_label]

  n = len(comparison_feature_space)
  distance = float('inf')
  most_similar_label = img_label
  distances = []
  for i in range(n):
    if i != img_label:
      distances.append({"label": i, "distance": math.dist(comparison_vector, comparison_feature_space[i])})

  distances = sorted(distances, key=lambda x: x["distance"], reverse=False)[:knum]

  for img in distances:
    print(img)

In [42]:
match selected_latent_space:

  case "" | "image_sim":
    
    extract_similarities_ls1_ls4(selected_latent_space, selected_dim_reduction_method, selected_feature_model, data, image_id)

  case "label_sim":

    extract_similarities_ls3(selected_dim_reduction_method, data, image_id)

  case "cp":

    extract_similarities_ls2(data, image_id)


{'label': 19, 'distance': 6.355424425784916e-06}
{'label': 33, 'distance': 1.2740918489579356e-05}
{'label': 54, 'distance': 6.85116641512525e-05}
{'label': 49, 'distance': 8.314856743319156e-05}
{'label': 28, 'distance': 0.00016883047586624644}
{'label': 36, 'distance': 0.0002122873813686887}
{'label': 35, 'distance': 0.00023350326417372972}
{'label': 44, 'distance': 0.0002406295461878455}
{'label': 9, 'distance': 0.00024234313765943418}
{'label': 10, 'distance': 0.0002640071831902495}
